## **Faiss** (Facebook AI Similarity Search) 
is a library for **efficient similarity search and clustering of dense vectors**, mainly used to **quickly find similar items** (like text or images) based on their **vector embeddings**.

In [1]:
# for Data ingestion
from langchain_community.document_loaders import TextLoader
# for Data splitting
from langchain_text_splitters import RecursiveCharacterTextSplitter
# for Creating the embeddings
from langchain.embeddings import OllamaEmbeddings
# For storing the embeddigs
from langchain.vectorstores import FAISS

# Data Ingestion and splitting data into document of chunks

In [2]:
loader=TextLoader("speech.txt")
docs=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

final_docs=text_splitter.split_documents(docs)
final_docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(metadata={'source': 'speech.txt'}, page_content

# Creating embeddings and storing them in Faiss Db 

In [3]:
embeddings=OllamaEmbeddings(model="deepseek-r1")

vector_store=FAISS.from_documents(final_docs, embeddings)

#vector_store.save_local("faiss_index")

vector_store

C:\Users\zaynb\AppData\Local\Temp\ipykernel_17080\2555306606.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings=OllamaEmbeddings(model="deepseek-r1")


# Querying in Faiss Db using similarity search

In [4]:
## querying the vector store
query="what is the desired outcome of the war?"
docs=vector_store.similarity_search(query, k=3)
for doc in docs:
    print(doc.page_content)

always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.
and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.
…


# RETRIVERS

**A retriever in LangChain is a tool that finds and returns the most relevant pieces of information (text chunks) from a data source based on a user’s query.**

we can also convert vector stores in into a Retriver class.This allows us to easily use it in other langchain methods,which largely work with retrivers

In [5]:
retriver=vector_store.as_retriever()
docs=retriver.invoke(query)
for doc in docs:
    print(doc.page_content)

always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.
and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.
…
Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.


### What’s the difference?


1. **Using vector store directly:**
```python
vector_store.similarity_search(query, k=3)
```
👉 This just searches and gives back top 3 similar texts.  
You control everything manually.

---

2. **Using a retriever:**
```python
retriever = vector_store.as_retriever()
retriever.invoke(query)
```
👉 This is like **wrapping your vector store** in a smarter tool.

---

### 💡 Why turn vector store into a retriever?

Because retrievers:
- Are **easier to plug into LangChain pipelines** (like chatbots or Q&A systems).
- Can **do more advanced stuff** like filtering, scoring, etc.
- Make your code **cleaner and future-ready**.

---

### ✅ Simple Summary:

| Method         | What it does                           |
|----------------|----------------------------------------|
| `similarity_search()` | Basic search, you do everything yourself |
| `as_retriever()`      | Smart wrapper, works better with LangChain tools |

# similarity search with score
lower score is given higher priority

In [6]:
vector_store.similarity_search_with_score(query, k=3)

[(Document(id='7457257d-3eb2-4c25-bd40-2b3ee09c2c1e', metadata={'source': 'speech.txt'}, page_content='always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  np.float32(15794.816)),
 (Document(id='c7f3acde-5e9c-4c16-b338-60e9a81fcffa', metadata={'source': 'speech.txt'}, page_content='and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
  np.float32(16349.354)),
 (Document(id='94bbf572-4caf-4ef5-ba14-18ab5b6085f8', metadata={'source': 'speech.txt'}, page_content='…'),
  np.float32(18679.725))]

# similarity search by vector

In [7]:
embedded_vectors=embeddings.embed_query(query)
vector_store.similarity_search_by_vector(embedded_vectors, k=3)

[Document(id='7457257d-3eb2-4c25-bd40-2b3ee09c2c1e', metadata={'source': 'speech.txt'}, page_content='always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='c7f3acde-5e9c-4c16-b338-60e9a81fcffa', metadata={'source': 'speech.txt'}, page_content='and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='94bbf572-4caf-4ef5-ba14-18ab5b6085f8', metadata={'source': 'speech.txt'}, page_content='…')]

# Storing the vector store in a local directory

In [8]:
vector_store.save_local("faiss_index")

# Loading the vector store and retrieving the data

In [10]:
new_vector_store=FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)
new_vector_store.similarity_search(query, k=3)

[Document(id='7457257d-3eb2-4c25-bd40-2b3ee09c2c1e', metadata={'source': 'speech.txt'}, page_content='always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='c7f3acde-5e9c-4c16-b338-60e9a81fcffa', metadata={'source': 'speech.txt'}, page_content='and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='94bbf572-4caf-4ef5-ba14-18ab5b6085f8', metadata={'source': 'speech.txt'}, page_content='…')]

| Feature                         | **Chroma**                                        | **FAISS**                                         |
|----------------------------------|---------------------------------------------------|---------------------------------------------------|
| **Definition**                   | An open-source embedding database                 | Facebook AI Similarity Search (FAISS)             |
| **Type**                         | Vector database with persistent storage option    | In-memory vector search library                   |
| **Persistence**                 | ✅ Supports persistent storage                    | ❌ No built-in persistence (unless manually saved) |
| **Speed**                        | Fast for local usage                              | Very fast and optimized for large-scale search    |
| **Installation**                 | `pip install chromadb`                            | `pip install faiss-cpu` (or `faiss-gpu`)          |
| **Ease of Use**                 | Easy, beginner-friendly                           | Requires more setup (especially for persistence)  |
| **Best Use Case**               | Quick setup, local dev, and small-medium data     | Large-scale, high-performance similarity search   |
| **LangChain Integration**       | Supported as `Chroma` vector store                | Supported as `FAISS` vector store                 |
| **Storage Type**                | Uses SQLite or DuckDB for local DB                | Stores data in RAM (can be saved manually)        |

### Summary:
- **Chroma**: Great for simple local projects with built-in persistence.
- **FAISS**: Ideal for high-performance and large-scale vector similarity search but requires more manual handling (especially for saving/loading)